This file contains all the solution solvers:

* Bayesian optimization

* Random embedding Bayesian optimization



Some functions used


### Normalization ( scale variable from [min, max] to [0,1] ):
a = (b-min)/(max-min) 

### de-normalization ( scale from [0,1] to [min, max] ):
b = a*(max-min) + min

### scale variable:
scale $a\in [a_l,a_u]$ to $b\in [b_l,b_u]$

$b = \frac{a-a_l}{(a_u-a_l)}(b_u-b_l)+b_l$

### standardization (scale to be normally distributed N(0,1) ):
x with $\mu, \sigma$

$y\sim N(0.1)$, $y = \frac{x-\mu}{\sigma}$



# Bayesian optimization

In [12]:
# generate y, with 0
def gen_high_dimension_variable(nn): ## nn: number of random y, nn<=10, need to adjust 100 to big value
    rand_yy = []
    rand_yy.append([0] * dim)# add 0 list
    #rand_yy.append([int(i/2) for i in x_real_u_bound.tolist()])
    for i in range (nn - 1): #(nn-1):
        test_y = np.random.uniform(x_real_l_bound.tolist(), x_real_u_bound.tolist()).astype(int) #np.random.randint(0,x_real_u_bound.tolist()) #torch.FloatTensor(low_dim).uniform_(y_l_bound, y_u_bound).to(ddtype)
        rand_yy.append(test_y)
    return torch.FloatTensor(rand_yy).to(ddtype)

In [4]:
# define collect initial points

def generate_initial_data_BO(n):  # n is number of initial value want to generate
    train_x = gen_high_dimension_variable(n)
    exact_obj = Optimization_function(train_x).unsqueeze(-1).to(ddtype)
    best_observation_value = exact_obj.max().item()
    best_observation_x = train_x[exact_obj.argmax().item()]

    return train_x,exact_obj, best_observation_value,  best_observation_x #train_x.float()

In [3]:
from botorch.models import SingleTaskGP, ModelListGP,FixedNoiseGP
from botorch import fit_gpytorch_model
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition.analytic import ExpectedImprovement,UpperConfidenceBound,ProbabilityOfImprovement, PosteriorMean
from botorch.optim import optimize_acqf, optimize_acqf_cyclic
from botorch.optim.fit import fit_gpytorch_torch
from botorch.utils.transforms import normalize, unnormalize, standardize


# one step of iteration (GP and acquisition), and find the next point

def get_next_points_BO(init_x, init_y, best_init_y, bounds_init_x, BATCH_SIZE):
    
    #global feed

    BO_bound = (torch.tensor([[0],[1]])).repeat(1,dim).to(ddtype)
    
    norm_init_x = normalize(init_x, bounds=bounds_init_x) # normalize x into [0,1]
    
    mean_init_y = init_y.mean().item() # mean of y
    std_init_y = init_y.std().item()  # std of y
    
    norm_init_y = (init_y-mean_init_y)/std_init_y  # standardize y    
    norm_init_Y_var = torch.full_like(norm_init_y, y_variance) # y with noise    
    norm_best_init_y = norm_init_y.max().item() # best stardized y
    #print(norm_init_x,norm_init_y,norm_init_Y_var)
    
    single_model = FixedNoiseGP(norm_init_x,norm_init_y,norm_init_Y_var) # define GP: single task homoskedastic exact GP
   
    mml = ExactMarginalLogLikelihood(single_model.likelihood,single_model) # define likelihood to fit GP:The exact marginal log likelihood (MLL) for an exact Gaussian process with a Gaussian likelihood.

    fit_gpytorch_model(mml) # Fit hyperparameters of a GPyTorch model, L-BFGS-B via scipy.optimize.minimize().
    #fit_gpytorch_model(mml, optimizer=fit_gpytorch_torch) # Fit hyperparameters of a GPyTorch model, line search

    EI = ExpectedImprovement(model = single_model,best_f = norm_best_init_y) # define acquisition function by GP

    #EI = UpperConfidenceBound(model = single_model) # define acquisition function by GP


    #print('1')
    norm_candidates, _ = optimize_acqf(
                    acq_function = EI,
                    bounds = BO_bound,
                    q = BATCH_SIZE,
                    num_restarts = 50,
                    raw_samples = 1000)
        
    candidates = unnormalize(norm_candidates, bounds=bounds_init_x)

    return candidates.int().to(ddtype)#, norm_candidates   # round to the lowest closet integer; change type back to 

In [ ]:
import time
import warnings
from botorch.exceptions import BadInitialCandidatesWarning


# define run BO
def BO_run(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_):
    
    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    verbose = False

    best_observed_all = []

    best_observed_all_x = []


    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):


        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        best_observed = []
        best_observed_x = []


        # generate initial training data and initialize model
        init_x, init_y, best_init_y, best_init_x = generate_initial_data_BO(N_initial)
        #best_observed_high = low_to_high_dimension(A,best_init_x)

        best_observed.append(best_init_y)
        best_observed_x.append(best_init_x)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    

            new_candidates = get_next_points_BO(init_x, init_y, best_init_y, x_real_bound, BATCH_SIZE)

            new_results = Optimization_function(new_candidates).unsqueeze(-1)

            init_x = torch.cat([init_x,new_candidates])
            init_y = torch.cat([init_y,new_results])

            best_init_y = init_y.max().item()
            best_init_x = init_x[init_y.argmax().item()]

            #print(new_results)

            best_observed.append(best_init_y)
            best_observed_x.append(best_init_x)

            if verbose:
                print(
                        f"\nBatch {iteration:>2}: best_value = "
                        f"({max(best_random):>4.2f}), "
                        f"time = {t1-t0:>4.2f}.", end=""
                    )
            else:
                print(".", end="")                    

        best_observed_all.append(best_observed)
        best_observed_all_x.append(best_observed_x)

    return best_observed_all,best_observed_all_x, init_x, init_y

In [ ]:
###### add more trial

# define BO add batch size (Follow up previous, run more iterations)
    
# run N_BATCH rounds of BayesOpt after the initial random batch

def BO_add_iter(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_,best_observed_all_,best_observed_all_x_, init_x_, init_y_,x_real_bound_):
    verbose = False

    best_init_y_ = init_y_.max().item()
    
    for iteration in range(1, N_BATCH + 1):    

        new_candidates = get_next_points_BO(init_x_, init_y_, best_init_y_, x_real_bound_, BATCH_SIZE_)
        new_results = daily_revenue_BOTorch(new_candidates).unsqueeze(-1)

        init_x_ = torch.cat([init_x_,new_candidates])
        init_y_ = torch.cat([init_y_,new_results])

        best_init_y = init_y_.max().item()
        best_init_x = init_x_[init_y_.argmax().item()]

        #print(new_results)

        best_observed_all_[0].append(best_init_y)
        best_observed_all_x_[0].append(best_init_x)

        if verbose:
            print(
                    f"\nBatch {iteration:>2}: best_value = "
                    f"({max(best_random):>4.2f}), "
                    f"time = {t1-t0:>4.2f}.", end=""
                )
        else:
            print(".", end="")                   
            
    return best_observed_all_,best_observed_all_x_, init_x_, init_y_

# Random Embedding Bayesian Optimization REMBO

In [ ]:
# Generate the projection matrix A as a (d x D) tensor
def gen_projection_rembo(d: int, D: int) -> torch.Tensor: #d low dimension, D high dimension
    AA = torch.randn( D,d, dtype=ddtype)
    return AA

In [2]:
# generate y in low dimension, with 0
def gen_low_dimension_variable(nn): ## nn: number of random y, nn<=10, need to adjust 100 to big value
    rand_yy = []
    rand_yy.append([0] * low_dim)# add 0 list
    for i in range (nn-1):#(nn):
        test_y = np.random.uniform(y_l_bound, y_u_bound,low_dim)#torch.FloatTensor(low_dim).uniform_(y_l_bound, y_u_bound).to(ddtype)
        rand_yy.append(test_y)
    return torch.FloatTensor(rand_yy).to(ddtype)

In [6]:
# convert low dimension y to high dimension x
def low_to_high_dimension(AA,yy): 
#yy is the low dimension variable, in how_dim
#AA is random embedding matrix, low_dim * dim
    scale_xx = torch.t(torch.matmul(AA,torch.t(yy)))
    # project to box bound of scale_x
    scale_xx = torch.clamp(scale_xx, min=x_l_bound, max=x_u_bound)
    scale_xx_ = (scale_xx-x_l_bound)/(x_u_bound-x_l_bound)
    #print(scale_xx_)
    real_xx = scale_xx_*(x_real_u_bound - x_real_l_bound) + x_real_l_bound #scale_xx_*x_real_u_bound
    #print(real_xx)
    return real_xx.int().to(ddtype)

In [ ]:
# define collect initial points

def generate_initial_data_REMBO(AA,n):  # n is number of initial value want to generate
    gen_low = gen_low_dimension_variable(n)
    train_x = low_to_high_dimension(AA,gen_low)
    exact_obj = Optimization_function(train_x).unsqueeze(-1).to(ddtype)
    best_observation_value = exact_obj.max().item()
    best_observation_low = gen_low[exact_obj.argmax().item()]

    return gen_low,exact_obj, best_observation_value,  best_observation_low #train_x.float()

In [2]:
from botorch.models import SingleTaskGP, ModelListGP,FixedNoiseGP
from botorch import fit_gpytorch_model
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition.analytic import ExpectedImprovement,UpperConfidenceBound,ProbabilityOfImprovement
from botorch.optim import optimize_acqf, optimize_acqf_cyclic
from botorch.optim.fit import fit_gpytorch_torch
from botorch.utils.transforms import normalize, unnormalize, standardize


# one step of iteration (GP and acquisition), and find the next point

def get_next_points_REMBO(init_x, init_y, best_init_y, bounds_init_x, BATCH_SIZE):

    BO_bound = (torch.tensor([[0],[1]])).repeat(1,low_dim).to(ddtype)
    
    norm_init_x = normalize(init_x, bounds=bounds_init_x) # normalize x into [0,1]
    
    mean_init_y = init_y.mean().item() # mean of y
    std_init_y = init_y.std().item()  # std of y
    
    norm_init_y = (init_y-mean_init_y)/std_init_y  # standardize y    
    norm_init_Y_var = torch.full_like(norm_init_y, y_variance) # y with noise    
    norm_best_init_y = norm_init_y.max().item() # best stardized y
    #print(norm_init_x,norm_init_y,norm_init_Y_var)
    
    single_model = FixedNoiseGP(norm_init_x,norm_init_y,norm_init_Y_var) # define GP: single task homoskedastic exact GP
   
    mml = ExactMarginalLogLikelihood(single_model.likelihood,single_model) # define likelihood to fit GP:The exact marginal log likelihood (MLL) for an exact Gaussian process with a Gaussian likelihood.

    fit_gpytorch_model(mml) # Fit hyperparameters of a GPyTorch model, L-BFGS-B via scipy.optimize.minimize().
    #fit_gpytorch_model(mml, optimizer=fit_gpytorch_torch) # Fit hyperparameters of a GPyTorch model, line search

    EI = ExpectedImprovement(model = single_model,best_f = norm_best_init_y) # define acquisition function by GP

    #EI = UpperConfidenceBound(model = single_model) # define acquisition function by GP


    #print('1')
    norm_candidates, _ = optimize_acqf(
                    acq_function = EI,
                    bounds = BO_bound,
                    q = BATCH_SIZE,
                    num_restarts = 50,
                    raw_samples = 1000)
        
    candidates = unnormalize(norm_candidates, bounds=bounds_init_x)

    return candidates.to(ddtype)#, norm_candidates   # round to the lowest closet integer; change type back to 

In [3]:
import warnings
from botorch.exceptions import BadInitialCandidatesWarning
import pickle


# define run rembo
def REMBO_run(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_):

    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    verbose = False

    best_observed_all = []

    best_observed_all_x = []

    A_all = []
    
    intermid = []


    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):

        A = gen_projection_rembo(low_dim,dim)

        A_all.append(A)


        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        best_observed = []
        best_observed_x = []


        # generate initial training data and initialize model
        init_x, init_y, best_init_y, best_init_x = generate_initial_data_REMBO(A,N_initial)
        #best_observed_high = low_to_high_dimension(A,best_init_x)

        best_observed.append(best_init_y)
        best_observed_x.append(best_init_x)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    

            new_candidates = get_next_points_REMBO(init_x, init_y, best_init_y, y_bound, BATCH_SIZE)

            new_high = low_to_high_dimension(A,new_candidates)

            new_results = Optimization_function(new_high).unsqueeze(-1)

            init_x = torch.cat([init_x,new_candidates])
            init_y = torch.cat([init_y,new_results])

            best_init_y = init_y.max().item()
            best_init_x = init_x[init_y.argmax().item()]

            #print(new_results)

            best_observed.append(best_init_y)
            best_observed_x.append(best_init_x)

            if verbose:
                print(
                        f"\nBatch {iteration:>2}: best_value = "
                        f"({max(best_random):>4.2f}), "
                        f"time = {t1-t0:>4.2f}.", end=""
                    )
            else:
                print(".", end="")                    

        best_observed_all.append(best_observed)
        best_observed_all_x.append(best_observed_x)
        
        # save intermediate files
        pickle.dump([best_observed_all,best_observed_all_x, init_x, init_y, A_all], open( 'result/intermid.p', "wb" ) )

    return best_observed_all,best_observed_all_x, init_x, init_y, A_all


In [2]:
from botorch.models import SingleTaskGP, ModelListGP,FixedNoiseGP
from botorch import fit_gpytorch_model
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition.analytic import ExpectedImprovement,UpperConfidenceBound,ProbabilityOfImprovement
from botorch.optim import optimize_acqf, optimize_acqf_cyclic
from botorch.optim.fit import fit_gpytorch_torch
from botorch.utils.transforms import normalize, unnormalize, standardize


# one step of iteration (GP and acquisition), and find the next point

def get_next_points_REMBO_ac(init_x, init_y, best_init_y, bounds_init_x, BATCH_SIZE,ac_function):

    BO_bound = (torch.tensor([[0],[1]])).repeat(1,low_dim).to(ddtype)
    
    norm_init_x = normalize(init_x, bounds=bounds_init_x) # normalize x into [0,1]
    
    mean_init_y = init_y.mean().item() # mean of y
    std_init_y = init_y.std().item()  # std of y
    
    norm_init_y = (init_y-mean_init_y)/std_init_y  # standardize y    
    norm_init_Y_var = torch.full_like(norm_init_y, y_variance) # y with noise    
    norm_best_init_y = norm_init_y.max().item() # best stardized y
    #print(norm_init_x,norm_init_y,norm_init_Y_var)
    
    single_model = FixedNoiseGP(norm_init_x,norm_init_y,norm_init_Y_var) # define GP: single task homoskedastic exact GP
   
    mml = ExactMarginalLogLikelihood(single_model.likelihood,single_model) # define likelihood to fit GP:The exact marginal log likelihood (MLL) for an exact Gaussian process with a Gaussian likelihood.

    fit_gpytorch_model(mml) # Fit hyperparameters of a GPyTorch model, L-BFGS-B via scipy.optimize.minimize().
    #fit_gpytorch_model(mml, optimizer=fit_gpytorch_torch) # Fit hyperparameters of a GPyTorch model, line search

    ###### define different acquisition functions
    #EI = ExpectedImprovement(model = single_model,best_f = norm_best_init_y) # define acquisition function by GP
    if ac_function == 'EI':
        EI = ExpectedImprovement(model = single_model,best_f = norm_best_init_y) # define acquisition function by GP
    elif ac_function == 'PI':
        EI = ProbabilityOfImprovement(model = single_model,best_f = norm_best_init_y)
    elif ac_function == 'UCB':
        EI = UpperConfidenceBound(model = single_model, beta = 0.2) # define acquisition function by GP
    elif ac_function == 'PM':
        EI = PosteriorMean(model= single_model)



    #print('1')
    norm_candidates, _ = optimize_acqf(
                    acq_function = EI,
                    bounds = BO_bound,
                    q = BATCH_SIZE,
                    num_restarts = 50,
                    raw_samples = 1000)
        
    candidates = unnormalize(norm_candidates, bounds=bounds_init_x)

    return candidates.to(ddtype)#, norm_candidates   # round to the lowest closet integer; change type back to 

In [3]:
import warnings
from botorch.exceptions import BadInitialCandidatesWarning
import pickle


# define run rembo
def REMBO_run_ac(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_,ac_function):
    global A_0, init_x_0, init_y_0, best_init_y_0, best_init_x_0 
    #print(A, init_x, init_y, best_init_y, best_init_x)
    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    verbose = False

    best_observed_all = []

    best_observed_all_x = []

    A_all = []
    
    intermid = []


    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):

        #A = gen_projection_rembo(low_dim,dim)

        A_all.append(A_0)


        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        best_observed = []
        best_observed_x = []


        # generate initial training data and initialize model
        init_x, init_y, best_init_y, best_init_x = init_x_0, init_y_0, best_init_y_0, best_init_x_0
        #best_observed_high = low_to_high_dimension(A,best_init_x)

        best_observed.append(best_init_y)
        best_observed_x.append(best_init_x)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    

            new_candidates = get_next_points_REMBO_ac(init_x, init_y, best_init_y, y_bound, BATCH_SIZE,ac_function)

            new_high = low_to_high_dimension(A_0,new_candidates)

            new_results = Optimization_function(new_high).unsqueeze(-1)

            init_x = torch.cat([init_x,new_candidates])
            init_y = torch.cat([init_y,new_results])

            best_init_y = init_y.max().item()
            best_init_x = init_x[init_y.argmax().item()]

            #print(new_results)

            best_observed.append(best_init_y)
            best_observed_x.append(best_init_x)

            if verbose:
                print(
                        f"\nBatch {iteration:>2}: best_value = "
                        f"({max(best_random):>4.2f}), "
                        f"time = {t1-t0:>4.2f}.", end=""
                    )
            else:
                print(".", end="")                    

        best_observed_all.append(best_observed)
        best_observed_all_x.append(best_observed_x)
        
        # save intermediate files
        pickle.dump([best_observed_all,best_observed_all_x, init_x, init_y, A_all], open( 'result/intermid.p', "wb" ) )

    return best_observed_all,best_observed_all_x, init_x, init_y, A_all


In [ ]:
########### add more trial

def REMBO_add_trail(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_):


    warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    verbose = False


    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):

        A = gen_projection_rembo(low_dim,dim)

        A_all.append(A)


        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        best_observed = []
        best_observed_x = []


        # generate initial training data and initialize model
        init_x, init_y, best_init_y, best_init_x = generate_initial_data_REMBO(A,N_initial)
        #best_observed_high = low_to_high_dimension(A,best_init_x)

        best_observed.append(best_init_y)
        best_observed_x.append(best_init_x)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    

            new_candidates = get_next_points_REMBO(init_x, init_y, best_init_y, y_bound, BATCH_SIZE)

            new_high = low_to_high_dimension(A,new_candidates)

            new_results = Optimization_function(new_high).unsqueeze(-1)

            init_x = torch.cat([init_x,new_candidates])
            init_y = torch.cat([init_y,new_results])

            best_init_y = init_y.max().item()
            best_init_x = init_x[init_y.argmax().item()]

            #print(new_results)

            best_observed.append(best_init_y)
            best_observed_x.append(best_init_x)

            if verbose:
                print(
                        f"\nBatch {iteration:>2}: best_value = "
                        f"({max(best_random):>4.2f}), "
                        f"time = {t1-t0:>4.2f}.", end=""
                    )
            else:
                print(".", end="")                    

        best_observed_all.append(best_observed)
        best_observed_all_x.append(best_observed_x)
    
    return best_observed_all,best_observed_all_x, init_x, init_y, A_all



# Random search

In [ ]:
# define collect initial points

def generate_initial_data_BO(n):  # n is number of initial value want to generate
    train_x = gen_high_dimension_variable(n)
    exact_obj = Optimization_function(train_x).unsqueeze(-1).to(ddtype)
    best_observation_value = exact_obj.max().item()
    best_observation_x = train_x[exact_obj.argmax().item()]

    return train_x,exact_obj, best_observation_value,  best_observation_x #train_x.float()

In [ ]:
# define run BO
def Random_search(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_):
       
    best_observed_all = []

    best_observed_all_x = []

    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):

        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        best_observed = []
        best_observed_x = []

        rand_x = [np.random.uniform(x_real_l_bound.tolist(), x_real_u_bound.tolist()).astype(int)]
        init_x = torch.FloatTensor(rand_x).to(ddtype)
        
        init_y = Optimization_function(init_x).unsqueeze(-1).to(ddtype)
        
        best_init_y = init_y.max().item()
        
        best_init_x = init_x[init_y.argmax().item()]
        
        best_observed.append(best_init_y)
        best_observed_x.append(best_init_x)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    
            
            rand_x_new = [np.random.uniform(x_real_l_bound.tolist(), x_real_u_bound.tolist()).astype(int)]
            init_x_new = torch.FloatTensor(rand_x_new).to(ddtype)
        
            init_y_new = Optimization_function(init_x_new).unsqueeze(-1).to(ddtype)

    
            init_x = torch.cat([init_x,init_x_new])
            init_y = torch.cat([init_y,init_y_new])

            best_init_y = init_y.max().item()
            best_init_x = init_x[init_y.argmax().item()]

            #print(new_results)

            best_observed.append(best_init_y)
            best_observed_x.append(best_init_x)

            print(".", end="")                    

        best_observed_all.append(best_observed)
        best_observed_all_x.append(best_observed_x)

    return best_observed_all,best_observed_all_x, init_x, init_y

In [17]:
# add iteration
def Random_add_iter(N_initial_,BATCH_SIZE_,N_BATCH_,N_TRIALS_,Run_result_):
       
    best_observed_all = Run_result[0]
    best_observed_all_x = Run_result[1]
    #init_x = Run_result[2]
    #init_y = Run_result[3]

    # average over multiple trials
    for trial in range(1, N_TRIALS + 1):

        print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
        best_observed = best_observed_all[trial - 1]
        best_observed_x = best_observed_all_x [trial - 1]

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    
            
            rand_x_new = [np.random.uniform(x_real_l_bound.tolist(), x_real_u_bound.tolist()).astype(int)]
            
            init_x_new = torch.FloatTensor(rand_x_new).to(ddtype)
            #print(rand_x_new)
            init_y_new = Optimization_function(init_x_new).unsqueeze(-1).to(ddtype)
            #print(init_y_new)
    
            if init_y_new >= best_observed[-1]:
                best_observed.append(init_y_new)
                best_observed_x.append(init_x_new)
            else:
                best_observed.append(best_observed[-1])
                best_observed_x.append(best_observed_x[-1])
          

            print(".", end="")                    

    return best_observed_all,best_observed_all_x, init_x, init_y